## Preparar el entorno y carga del modelo 

In [94]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Mapea índice -> categoría (lo que usaste en training)
index_to_category = {
    0: "INDUSTRIAL", 
    1: "CIVIL", 
    2: "INFORMÁTICA",
    3: "TELECOMUNICACIONES",
    4: "ARQUITECTURA",
    5: "FILOSOFÍA",
    6: "PSICOLOGÍA",
    7: "LETRAS",
    8: "COMUNICACIÓN SOCIAL",
    9: "EDUCACIÓN",
    10: "ADMINISTRACIÓN",
    11: "CONTADURÍA",
    12: "RELACIONES INDUSTRIALES",
    13: "SOCIOLOGÍA",
    14: "ECONOMÍA",
    15: "DERECHO",
    16: "TEOLOGÍA"
}


# Dispositivo (GPU si está disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cargar tokenizer
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Cargar modelo con la misma configuración
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=17
)
model.load_state_dict(torch.load('best_model_state.bin', map_location=device))
model.to(device)
model.eval()  # Modo inferencia

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\aleja\AppData\Local\Temp\ipykernel_16832\718828362.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this m

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [95]:
## predict carrer

def predict_career(text_list, tokenizer, model, max_length=128):
    """
    text_list: lista de strings con las respuestas de usuario
    tokenizer: tokenizer de Hugging Face
    model: modelo BertForSequenceClassification cargado
    """
    # Tokenizar
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [batch_size, num_labels]
    
    # Obtener predicciones
    preds = torch.argmax(logits, dim=1).cpu().numpy()  # Índices de clase
    predicted_labels = [index_to_category[p] for p in preds]
    return predicted_labels



def predict_career_top3(text_list, tokenizer, model, index_to_category, 
                        device, max_length=128, top_k=3, threshold=0.5):
    """
    text_list: lista de strings con las respuestas de usuario
    tokenizer: tokenizer de Hugging Face
    model: modelo BertForSequenceClassification cargado
    index_to_category: diccionario {índice: categoría}
    device: 'cuda' o 'cpu'
    max_length: longitud máxima de tokenización
    top_k: número de predicciones (top) a retornar
    threshold: umbral mínimo de probabilidad para asignar una clase (vs 'UNKNOWN')
    """
    # Tokenizar
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [batch_size, num_labels]

        # Convertir logits a probabilidades con softmax
        probs = torch.softmax(logits, dim=1)  # [batch_size, num_labels]
        
        # Obtener los top_k índices y probabilidades
        top_probs, top_indices = probs.topk(top_k, dim=1, largest=True, sorted=True)
        # top_probs, top_indices => ([batch_size, top_k]), ([batch_size, top_k])

    batch_results = []
    for i in range(len(text_list)):
        # Extraer las k probabilidades e índices para esta fila i
        row_probs = top_probs[i].cpu().numpy()
        row_indices = top_indices[i].cpu().numpy()

        result = []
        for idx, p in zip(row_indices, row_probs):
            category_name = index_to_category[idx]
            result.append((category_name, float(p)))
        batch_results.append(result)

    return batch_results

In [96]:
# Texto filtrado y concatenado
# Texto filtrado y concatenado
user_text = """Como desarrollador de IA, creo modelos para luego utilizarlos en aplicaciones. 
Me gustaba mucho la parte matemática, por ser algo práctico una vez entendías el tema. 
Sin embargo, también me gustaba la materia de literatura, historia y biología. 
Educación física, soy mala en los deportes. 
Si, un curso de inglés, me gusta mucho el idioma. 
Me gusta ver series y películas, también ver documentales o videos para aprender cosas nuevas de ciencia, historia, cultura general. 
Ciencia, tecnología, arte y cultura. 
Acerca de ciencia, como la teoría de la relatividad, cuántica, me gusta mucho entender el mundo desde esa perspectiva. 
Crear, investigar y resolver problemas. 
Me interesa entender cómo funciona el mundo, Me gusta imaginar y crear cosas nuevas, Prefiero resolver problemas prácticos y concretos. 
En un café. 
Tecnológico. 
Emprender."""

user_text = """Desde muy joven, he sentido una gran pasión por la tecnología. Me encanta aprender y desarrollar soluciones de software que resuelvan problemas reales; por ello, he orientado mis estudios hacia la Ingeniería Informática. Además, me fascina el mundo de las telecomunicaciones, ya que creo que conectar a las personas a través de redes modernas y eficientes es clave para el avance social y económico. Por otro lado, también me interesa la parte gerencial y organizativa, lo que me lleva a valorar la Administración; considero esencial saber planificar, gestionar proyectos y liderar equipos para llevar adelante iniciativas tecnológicas. En resumen, mi formación y mis intereses se centran en el desarrollo de sistemas informáticos, la conectividad a través de telecomunicaciones y la gestión estratégica en entornos empresariales."""
user_text = """Estoy interesado en Ingeniería Industrial. Fui becado y congelé mi carrera en 2020 por temas económicos. Me gustaban las materias de Matemática e Historia de Venezuela. No me agradaban Biología y Física. Me interesa resolver problemas y explorar culturas. Me gustaría ejercer mi carrera en España y viajar por el mundo."""



top3_preds = predict_career_top3(
    text_list=[user_text],
    tokenizer=tokenizer,
    model=model,
    index_to_category=index_to_category,
    device=device,
    top_k=3,
    threshold=0.5
)
print("Top 3 predicciones:", top3_preds[0])

Top 3 predicciones: [('FILOSOFÍA', 0.272817999124527), ('SOCIOLOGÍA', 0.11937537044286728), ('ECONOMÍA', 0.10283531993627548)]


In [97]:
# Respuesta del estudiante 1
user_text1 = """Estoy interesado en Ingeniería Industrial. Fui becado y congelé mi carrera en 2020 por temas económicos. Me gustaban las materias de Matemática e Historia de Venezuela. No me agradaban Biología y Física. Me interesa resolver problemas y explorar culturas. Me gustaría ejercer mi carrera en España y viajar por el mundo."""

# Respuesta del estudiante 2
user_text2 = """Estoy interesado en Ingeniería Informática. Me gustaban Matemáticas y Deportes, eran las que mejores se me daban. No me agradaba Historia porque tengo mala memoria. Me interesa crear, ayudar y resolver problemas. Mi objetivo es conseguir empleo remoto en el extranjero."""

# Respuesta del estudiante 3
user_text3 = """Estoy interesada en Psicología. Me gustaban Física y Biología porque las entendía. No me agradaban Matemáticas y Castellano porque me parecían tediosas. Me gusta leer, ver películas y bailar. Me interesa crear e investigar. Mi meta es viajar y entender mejor el mundo."""

# Respuesta del estudiante 4
user_text4 = """Estoy interesada en Ingeniería Informática. Me gustaban Matemáticas, Química, Física y Contabilidad porque me gustan las cosas exactas. No me agradaban Dibujo, Historia y Geografía porque había que hacer muchas maquetas. Me gusta liderar, enseñar y resolver problemas. Mi objetivo es trabajar en una oficina."""

# Respuesta del estudiante 5
user_text5 = """Estoy interesado en Ingeniería Informática. Me gustaban Historia, Física y Química porque se me hacían divertidas. No me agradaba Castellano porque me parece aburrida. Me gusta jugar videojuegos y leer. Me interesa resolver problemas. Mi meta es ganar dinero para ayudar a mis padres."""

# Respuesta del estudiante 6
user_text6 = """Estoy interesado en Ingeniería en Informática. Me gustaban Matemáticas por sus propiedades y formas interesantes y Química por las reacciones y experimentos. No me agradaba Geografía porque no soy bueno ubicándome. Me gusta crear, liderar, ayudar y resolver problemas. Mi meta es desarrollar videojuegos."""

# Respuesta del estudiante 7
user_text7 = """Estoy interesada en Ingeniería Informática. Me gustaban Matemática y Física porque los números siempre me han llamado la atención. No me agradaba Química porque no entendía nada. Me interesa crear y explorar temas psicológicos y emocionales. Mi objetivo es manejarme bien y cocinar."""

# Respuesta del estudiante 8
user_text8 = """Estoy interesado en Ingeniería Informática. Me gustaban Matemáticas, Historia y Artes porque me interesan los cálculos, el pasado y el arte. No me agradaban Química y Geografía porque no las entendía bien. Me gusta dibujar, leer manga y jugar videojuegos. Mi meta es crear cómics."""

# Respuesta del estudiante 9
user_text9 = """Estoy interesada en Relaciones Industriales. Me gustaban Biología y Química porque me agradan sus funciones. No me agradaba Educación Física porque siempre terminaba con dolor de cabeza. Me gusta bailar y explorar culturas. Mi meta es que mi emprendimiento de dulces llegue a más personas."""

# Respuesta del estudiante 10
user_text10 = """Estoy interesada en Comunicación Social. Me gustaba Biología porque me ayudó a entender procesos humanos. No me agradaba Química porque no tuve un docente permanente. Me interesa ayudar y crear. Mi meta es emprender proyectos propios."""

# Respuesta del estudiante 11
user_text11 = """Estoy interesado en Comunicación Social. Me gustaban las materias relacionadas con Historia porque investigaba por mi cuenta. No me agradaban las Ciencias porque me costaban bastante. Me gusta leer, editar, y hacer fotografía. Mi meta es crear cosas útiles y vivir de ello."""

# Respuesta del estudiante 12
user_text12 = """Estoy interesada en Contaduría Pública. Me gustaban Contabilidad y Diseño porque me desenvolvía mejor. No me agradaban Matemática, Historia, Geografía y Política porque no me interesaban. Me gusta hacer ejercicio, pintar y bailar. Mi meta es ser bailarina profesional."""

# Respuesta del estudiante 13
user_text13 = """Estoy interesada en Educación. Me gustaban Matemática, Química, Biología y Arte porque las entendía bien. No me agradaba Física porque me enredaba en pequeñeces. Me gusta bailar, leer y hornear. Mi meta es abrir una pastelería y presentarme como bailarina profesional."""

# Respuesta del estudiante 14
user_text14 = """Estoy interesada en Ingeniería Informática. Me gustaban Matemáticas y Física porque eran interesantes. No me agradaba Química porque no me llamaba la atención. Me interesa resolver problemas y enseñar. Mi meta es levantar una empresa de desarrollo."""

# Respuesta del estudiante 15
user_text15 = """Estoy interesado en Ingeniería Informática. Me gustaba Computación porque me encantan las computadoras. No me agradaba Castellano porque no me gustan los idiomas. Me gusta leer y jugar videojuegos. Mi meta es tener mi propia oficina con un jardín."""

# Respuesta del estudiante 16
user_text16 = """Estoy interesado en Ingeniería en Sistemas. Me gustaba Química porque el profesor hacía los temas dinámicos. No me agradaba Historia de Venezuela porque era monótona. Me interesa enseñar y ayudar. Mi meta es aprender alemán y ser políglota."""

# Respuesta del estudiante 17
user_text17 = """Estoy interesado en Ingeniería Informática. Me gustaban Biología y Psicología porque me gusta entender el comportamiento. No me agradaba Química porque me costaba. Me gusta estar con mis amigos. Mi meta es conseguir estabilidad económica y aprender a tocar la guitarra."""

# Respuesta del estudiante 18
user_text18 = """Estoy interesada en Ingeniería Informática. Me gustaba Matemática porque es práctica y me gustan temas de ciencia. No me agradaba Educación Física porque soy mala en deportes. Me gusta ver series y aprender cosas nuevas. Mi meta es emprender."""

# Respuesta del estudiante 19
user_text19 = """Estoy interesada en Comunicación Social. Me gustaban Inglés, Biología y Psicología porque me sentía conectada con ellas. No me agradaban Física, Química y Matemática porque me costaba entenderlas. Me gusta investigar y enseñar. Mi meta es fundar una ONG para ayudar a mujeres y niños."""

# Respuesta del estudiante 20
user_text20 = """Estoy interesado en Física. Me gustaban Matemáticas y Física porque me gustan los cálculos. No me agradaba Dibujo Técnico porque no tenía paciencia. Me interesa crear y resolver problemas. Mi meta es graduarme de la universidad."""


In [98]:
# // ... existing code ...

import spacy
import re

# Cargar el modelo de SpaCy para español
nlp = spacy.load("es_core_news_sm")

def preprocess_text(text: str) -> str:
    """
    Preprocesa el texto aplicando los siguientes pasos:
    1. Convierte a minúsculas
    2. Elimina caracteres especiales manteniendo acentos
    3. Elimina espacios extra
    4. Elimina stopwords
    5. Aplica lematización
    
    Args:
        text (str): Texto a preprocesar
        
    Returns:
        str: Texto preprocesado
    """
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar caracteres especiales pero mantener acentos
    text = re.sub(r'[^a-záéíóúñü\s]', '', text)
    
    # Reemplazar múltiples espacios por uno solo
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    # Procesar con SpaCy para lematización y eliminación de stopwords
    doc = nlp(text)
    
    # Unir los lemas de las palabras que no son stopwords ni puntuación
    cleaned_text = " ".join([token.text for token in doc if not token.is_punct])  # No eliminar stopwords ni lematizar todo

    
    return cleaned_text

def predict_career_with_preprocessing(text_list: list, tokenizer, model, 
                                   index_to_category: dict, device: str,
                                   max_length: int = 128, top_k: int = 3, temperature: float = 1.5) -> list:
    """
    Preprocesa el texto y realiza la predicción de carreras.
    
    Args:
        text_list (list): Lista de textos a procesar
        tokenizer: Tokenizer de BERT
        model: Modelo BERT entrenado
        index_to_category (dict): Mapeo de índices a nombres de carreras
        device (str): Dispositivo para procesamiento ('cuda' o 'cpu')
        max_length (int): Longitud máxima de tokens
        top_k (int): Número de predicciones top a retornar
        
    Returns:
        list: Lista de tuplas (carrera, probabilidad) para cada texto
    """
    # Preprocesar cada texto
    processed_texts = [preprocess_text(text) for text in text_list]
    
    # Tokenizar
    inputs = tokenizer(
        processed_texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Aplicar temperatura para suavizar las probabilidades
        scaled_logits = logits / temperature
        probs = torch.softmax(scaled_logits, dim=1)
        
        top_probs, top_indices = probs.topk(top_k, dim=1, largest=True, sorted=True)


    batch_results = []
    for i in range(len(text_list)):
        row_probs = top_probs[i].cpu().numpy()
        row_indices = top_indices[i].cpu().numpy()

        result = []
        for idx, p in zip(row_indices, row_probs):
            category_name = index_to_category[idx]
            result.append((category_name, float(p)))
        batch_results.append(result)

    return batch_results

# Ejemplo de uso:
user_text = """Como desarrollador de IA, creo modelos para luego utilizarlos en aplicaciones. 
Me gustaba mucho la parte matemática, por ser algo práctico una vez entendías el tema. 
Sin embargo, también me gustaba la materia de literatura, historia y biología. 
Educación física, soy mala en los deportes. 
Si, un curso de inglés, me gusta mucho el idioma. 
Me gusta ver series y películas, también ver documentales o videos para aprender cosas nuevas de ciencia, historia, cultura general. 
Ciencia, tecnología, arte y cultura. 
Acerca de ciencia, como la teoría de la relatividad, cuántica, me gusta mucho entender el mundo desde esa perspectiva. 
Crear, investigar y resolver problemas. 
Me interesa entender cómo funciona el mundo, Me gusta imaginar y crear cosas nuevas, Prefiero resolver problemas prácticos y concretos. 
En un café. 
Tecnológico. 
Emprender."""

# user_text = """Desde muy joven, he sentido una gran pasión por la tecnología. Me encanta aprender y desarrollar soluciones de software que resuelvan problemas reales; por ello, he orientado mis estudios hacia la Ingeniería Informática. Además, me fascina el mundo de las telecomunicaciones, ya que creo que conectar a las personas a través de redes modernas y eficientes es clave para el avance social y económico. Por otro lado, también me interesa la parte gerencial y organizativa, lo que me lleva a valorar la Administración; considero esencial saber planificar, gestionar proyectos y liderar equipos para llevar adelante iniciativas tecnológicas. En resumen, mi formación y mis intereses se centran en el desarrollo de sistemas informáticos, la conectividad a través de telecomunicaciones y la gestión estratégica en entornos empresariales."""

#user_text = """Estoy interesado en Ingeniería Industrial. Fui becado y congelé mi carrera en 2020 por temas económicos. Me gustaban las materias de Matemática e Historia de Venezuela. No me agradaban Biología y Física. Me interesa resolver problemas y explorar culturas. Me gustaría ejercer mi carrera en España y viajar por el mundo."""

# Realizar predicción con el texto preprocesado
predictions = predict_career_with_preprocessing(
    text_list=[user_text],
    tokenizer=tokenizer,
    model=model,
    index_to_category=index_to_category,
    device=device,
    top_k=3,
)

print("Texto original:")
print(user_text)
print("\nTexto preprocesado:")
print(preprocess_text(user_text))
print("\nPredicciones:")
for career, prob in predictions[0]:
    print(f"{career}: {prob:.2%}")

Texto original:
Como desarrollador de IA, creo modelos para luego utilizarlos en aplicaciones. 
Me gustaba mucho la parte matemática, por ser algo práctico una vez entendías el tema. 
Sin embargo, también me gustaba la materia de literatura, historia y biología. 
Educación física, soy mala en los deportes. 
Si, un curso de inglés, me gusta mucho el idioma. 
Me gusta ver series y películas, también ver documentales o videos para aprender cosas nuevas de ciencia, historia, cultura general. 
Ciencia, tecnología, arte y cultura. 
Acerca de ciencia, como la teoría de la relatividad, cuántica, me gusta mucho entender el mundo desde esa perspectiva. 
Crear, investigar y resolver problemas. 
Me interesa entender cómo funciona el mundo, Me gusta imaginar y crear cosas nuevas, Prefiero resolver problemas prácticos y concretos. 
En un café. 
Tecnológico. 
Emprender.

Texto preprocesado:
desarrollador ia modelo utilizar él aplicación gustar matemático práctico entendía tema gustar materia literatu